In [2]:
import collections
from imp import reload

import pandas as pd
import numpy as np




In [6]:
from tqdm import tqdm

user_id = np.random.randint(1,200,size=(2000,1))
click_article_id = np.random.randint(1,1000,size=(2000,1))
click_timestamp = np.random.randint(100000,999999,size=(2000,1))
created_at_ts = click_article_id*333%100
words_count = click_article_id*137%500
data= np.concatenate([user_id,click_article_id,click_article_id%8,click_timestamp,created_at_ts,words_count],axis=-1)
df_click = pd.DataFrame(data,columns=["user_id",'click_article_id','category_id','click_timestamp','created_at_ts','words_count'])
item_info_df = df_click
item_info_df = item_info_df.drop(['user_id','click_timestamp'],axis=1).drop_duplicates(subset=['click_article_id','category_id','created_at_ts'],keep='first')


item_emb_df = pd.DataFrame()
item_emb_df['article_id'] = item_info_df['click_article_id']
for i in range(0,250):
    emb = 'emb'+(str)(i)
    item_emb_df[emb] = np.random.randint(1,10,size=(len(item_emb_df),1))

C:\Users\lonless\AppData\Local\Temp/ipykernel_3920/2047912565.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider using pd.concat instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  item_emb_df[emb] = np.random.randint(1,10,size=(len(item_emb_df),1))


In [5]:
# 文章索引与文章id的字典映射
item_idx_2_rawid_dict = dict(zip(item_emb_df.index, item_emb_df['article_id']))

item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)
# 向量进行单位化
item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)




['2021.7.6 学习会.docx',
 'Clash for Windows.lnk',
 'Clash.for.Windows-0.15.2-win',
 'desktop.ini',
 'docs.txt.txt',
 'GitHub Desktop.lnk',
 'HEU23_Debug.txt',
 'IMG_0732(20201222-205924).JPG',
 'Microsoft Edge.lnk',
 '__MACOSX',
 '图片1.png',
 '新建文件夹',
 '新建文本文档.txt',
 '网安院宣传部副部梁小龙.docx']

In [57]:
#!/usr/bin/python
# -*- coding: utf-8 -*-
import numpy
import os
import sys
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer



def i2i_sim():

    #sys.setdefaultencoding('utf8')了 网易 杭研 大厦","小明 硕士 毕业 与 中国 科学院","我 爱 北京 天安门"]
    trainfile = open("C:\\Users\\lonless\\Desktop\\docs.txt","r",encoding='utf-8') #不同的documents用换行符隔开
    traincorpus = trainfile.readlines()


    #corpus=["我 来到 北京 清华大学","我 他 来到
    trainfile.close()
    corpus = traincorpus

    vectorizer=CountVectorizer()#该类会将文本中的词语转换为词频矩阵，矩阵元素a[i][j] 表示j词在i类文本下的词频
    tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, #max_features=n_features,
                                       stop_words='english')
    transformer=TfidfTransformer()#该类会统计每个词语的tf-idf权值
    tfidf=transformer.fit_transform(tfidf_vectorizer.fit_transform(corpus))#第一个fit_transform是计算tf-idf，第二个fit_transform是将文本转为词频矩阵
    word=tfidf_vectorizer.get_feature_names()#获取词袋模型中的所有词语
    weight=tfidf.toarray()#将tf-idf矩阵抽取出来，元素a[i][j]表示j词在i类文本中的tf-idf权重

    SimMatrix = (tfidf * tfidf.T).A
    #print(SimMatrix[1,3]) #"第一篇与第4篇的相似度"

    numpy.savetxt(r"C:\Users\lonless\Desktop\SimMatrix.csv", SimMatrix, delimiter=",") #保存相似度矩阵

In [55]:
SimMatrix[10,20]

0.0

In [52]:
topk = 10
item_idx_2_rawid_dict = dict(zip(item_emb_df.index, item_emb_df['article_id']))

item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)
# 向量进行单位化
item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)



sim, idx = SimMatrix.search(topk)

AttributeError: 'numpy.ndarray' object has no attribute 'search'

In [ ]:
item_sim_dict = collections.defaultdict(dict)
    for target_idx, sim_value_list, rele_idx_list in tqdm(zip(range(len(item_emb_np)), sim, idx)):
        target_raw_id = item_idx_2_rawid_dict[target_idx]
        # 从1开始是为了去掉商品本身, 所以最终获得的相似商品只有topk-1
        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]):
            rele_raw_id = item_idx_2_rawid_dict[rele_idx]
            item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value

    # 保存i2i相似度矩阵
    pickle.dump(item_sim_dict, open(save_path + 'emb_i2i_sim.pkl', 'wb'))